In [1]:

!pip uninstall -y numpy scipy scikit-learn torch torchvision torchaudio tensorboard tensorflow keras matplotlib

!pip install numpy==1.21.6

!pip install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1
!pip install transformers==4.25.1 accelerate bitsandbytes
!pip install datasets==2.21.0 huggingface-hub==0.34.0 
!pip install groq spacy nltk tiktoken

!python -m spacy download en_core_web_sm
!git clone https://github.com/ilatims-b/LLMLingua.git

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: scipy 1.15.3
Uninstalling scipy-1.15.3:
  Successfully uninstalled scipy-1.15.3
Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: tensorboard 2.18.0
Uninstalling tensorboard-2.18.0:
  Successfully uninstalled tensorboard-2.18.0
Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.1

In [2]:
#not needed if not using groq
!pip install groq

In [3]:
from datasets import load_dataset
import json
import os

In [4]:
data = []
NUM_SAMPLES = 1
datasets = ['multifieldqa_en', 'triviaqa']
current_idx = 0  # Running index across datasets

for dataset_ in datasets:
    data_ = load_dataset('THUDM/LongBench', dataset_, split='test', trust_remote_code=True)
    
    for idx, instance in enumerate(data_):
        if idx >= NUM_SAMPLES:
            break
        
        temp = {}
        temp["dataset"] = dataset_
        temp["idx"] = current_idx  # Use running index here instead of idx
        temp["prompt"] = instance['context'] + '\n' + instance['input']
        temp["answer"] = instance['answers']
        temp["length"] = instance['length']
        data.append(temp)
        
        current_idx += 1  # Increment the running index
        
os.makedirs("/kaggle/working/dataset", exist_ok=True)
with open("/kaggle/working/dataset/longbench_1.json", "w") as f:
    json.dump(data, f, indent=4)



Generating test split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:

os.chdir('/kaggle/working/LLMLingua/experiments/llmlingua2/data_collection')

In [ ]:
!pip install openai==0.28#not needed if not using gpt4

In [6]:

#generating compressed prompts from gpt4 for data annotation
!python compress.py --load_origin_from /kaggle/working/dataset/longbench_1.json \
--chunk_size 512 \
--compressor groq\
--model_name openai/gpt-oss-20b \
--api_key "your api key :)"\
--save_path /kaggle/working/dataset/longbench_1_compressed.json

num data: 2
You are an excellent linguist and very good at compressing passages into short expressions by removing unimportant words, while retaining as much information as possible.
Compress some text to short expressions, and such that you (GPT-4) can reconstruct it as close as possible to the original. Unlike the usual text compression, I need you to comply with the 5 conditions below: 1. You can ONLY remove unimportant words. 2. Do not change the order of words. 3. Do not change the original words, e.g. 'asking'->'ask' is NOT OK, 'current'->'now' is NOT OK. 4. Do not use abbreviations or emojis, e.g. 'without'->'w/o' is NOT OK, 'as soon as possible'->'ASAP' is NOT OK. 5. Do not add new words or symbols, this is very important. For example, 'dedicate 3 hours to each chapter'->'3 hours/chapter' is NOT OK because you add new token '/', just compress it into '3 hours each chapter'. '30 eggs plus 20 eggs equals 50 eggs'->'30+20=50' is also NOT OK becuase you add new symbols + and =, jus

In [7]:
os.makedirs("/kaggle/working/dataset/longbench_hf_dataset", exist_ok=True)
json.dump(list(json.load(open("/kaggle/working/dataset/longbench_1_compressed.json")).values()), open("/kaggle/working/dataset/longbench_hf_dataset/train.json", "w"), ensure_ascii=False)

In [8]:
#train data annotations
!python label_word.py \
--load_prompt_from /kaggle/working/dataset/longbench_hf_dataset  \
--window_size 400 \
--save_path /kaggle/working/dataset/longbench_1_labelled.json

Generating train split: 2 examples [00:00, 57.04 examples/s]
15it [00:01, 13.07it/s]
window size: 400, comp rate: 0.14955640453642954, hitting_rate: 0.1469762469261397, retrieval rate: 0.14175059371948887


In [9]:
#train data filtering, they have defined various metrics in paper
!python filter.py \
--load_path /kaggle/working/dataset/longbench_1_labelled.pt\
--save_path /kaggle/working/dataset/longbench_1_filtered.pt

15


In [10]:

os.chdir('/kaggle/working/LLMLingua/experiments/llmlingua2/model_training')

In [11]:
!pip install --force-reinstall tensorboard==2.13.0

  Using cached numpy-2.3.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.1/216.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 110.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 6.9 MB/s eta 0:00:00
Using cached numpy-2.3.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 76.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.6 M

In [12]:
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter

print(f"✅ NumPy: {np.__version__}")

print(f"✅ PyTorch: {torch.__version__}")
print("✅ All imports successful!")

✅ NumPy: 1.26.4
✅ PyTorch: 2.8.0+cu128
✅ All imports successful!


In [14]:
os.makedirs("/kaggle/working/roberta_custom", exist_ok=True)


In [15]:
!python train_roberta.py \
    --data_path /kaggle/working/dataset/longbench_1_filtered.pt \
    --save_path /kaggle/working/roberta_custom \
    --num_epoch 3 \
    --lr 3e-5 \
    --quantization float16 \
    --batch_size 16


CUDA test successful on cuda
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading model with float16 precision
Some weights of the model checkpoint at FacebookAI/xlm-roberta-large were not used when initializing XLMRobertaForTokenClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly ident